In [1]:
# EconWebscrape.ipynb
'''
    Author: Kevin Yao (email: kevinxy00@gmail.com)

    Purpose: To get abstracts from the top 10 Economics Journals,
    in terms of impact factors for the last 10 years. 
    To accomplish this, I will webscrape 
    https://ideas.repec.org/top/top.journals.simple10.html
    for links to each journal stored on the site. 
'''

# Dependencies
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

In [2]:
# Simple Impact Factors (Last 10 Years) for Journals:
''' gets the html from ideas.repec.org
    This webpage contains the link to each journals,
    which we will need to get the abstracts 
'''

base_url = "https://ideas.repec.org"
journs_route = "/top/top.journals.simple10.html"
url = base_url + journs_route

r = requests.get(url)
data = r.text # turns response into texts
soup = BeautifulSoup(data, "html.parser") # changes the response from text to html

In [3]:
soup_topList = soup.find(class_="toplist")
soup_href = soup_topList.find_all("a", href=True)


In [11]:
# get list of links to the top 10 journals 
''' 
NOTE: (includes occasional multiple links to same journal at different times)
# e.g. https://ideas.repec.org/s/wly/emetrp.html (Econometrica from sep. 2014 to present)
vs https://ideas.repec.org/s/ecm/emetrp.html (Econometrica from 1950 to Nov. 2013)
'''
number = 0
journ_ls = []
for entry in range(0, 14):
    number = number + 1
    base_link = soup_href[entry]["href"]
    url = base_url + base_link
    print(number, url)
    journ_ls.append(url)
    


1 https://ideas.repec.org/s/oup/qjecon.html
2 https://ideas.repec.org/s/aea/jeclit.html
3 https://ideas.repec.org/s/aea/aejmac.html
4 https://ideas.repec.org/s/wly/emetrp.html
5 https://ideas.repec.org/s/ecm/emetrp.html
6 https://ideas.repec.org/s/aea/aejapp.html
7 https://ideas.repec.org/s/ucp/jpolec.html
8 https://ideas.repec.org/s/oup/restud.html
9 https://ideas.repec.org/s/bla/jfinan.html
10 https://ideas.repec.org/s/oup/ecpoli.html
11 https://ideas.repec.org/s/bla/ecpoli.html
12 https://ideas.repec.org/s/oup/jeurec.html
13 https://ideas.repec.org/s/bla/jeurec.html
14 https://ideas.repec.org/s/tpr/jeurec.html


In [4]:
''' 
    Function for getting a list of links to journal articles 
    from the first page of a journal
'''
def get_article_links(url):
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")

    # page is broken up into lists for each volume/issue
    # this gets the raw text of each page
    soup_issues = soup.find_all(class_="list-group paperlist")
    
    # this converts raw text into links by finding href
    number = 0
    issues_link_ls = []
    for issue in soup_issues:
        soup_issues_link = issue.find_all("a", href=True)
        for link in soup_issues_link:
            number = number + 1
            base_link = link["href"]
            url = base_url + base_link
            print(number, url)
            issues_link_ls.append(url)

    return issues_link_ls


In [ ]:
''' 
# Test for above section: for number 1 article (journal of quarterly economics),
# get list of all hrefs
number = 0
issues_link_ls = []
for issue in soup_issues:
    soup_issues_link = issue.find_all("a", href=True)
    for link in soup_issues_link:
        number = number + 1
        base_link = link["href"]
        url = base_url + base_link
        print(number, url)
        issues_link_ls.append(url)
'''

In [ ]:
'''
# preliminary TEST for TEST: get abstract 
r = requests.get(issues_link_ls[0])
data = r.text
soup = BeautifulSoup(data, "html.parser")
soup_abstract = soup.find(id = "abstract-body")
print(soup_abstract.text)
'''

In [ ]:
'''
# TEST for getting the abstract from each issues link for the first journal
abstract_ls = []
for url in issues_link_ls:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    soup_abstract = soup.find(id = "abstract-body")
    abstract_ls.append(soup_abstract.text)
    # sleep for 1 second in between requests so webpage doesn't hate me too much
    time.sleep(1) 
abstract_ls
'''    


In [5]:
# PUTTING IT ALL TOGETHER

''' For each journal in the journal list of the top 10 journals, get list of url of articles
    for the first page of each journal. Then using those url, get the abstract for each article.
    Save to a list
'''
def get_abstracts(ls):
    abstract_ls = []
    journ_count = 0
    art_count = 0
    for journal_url in ls:
        article_links_ls = get_article_links(journal_url)
        journ_count = journ_count + 1
        print("****** Processing Journal # " + str(journ_count) + " ******")
        for url in article_links_ls:
            art_count = art_count + 1
            print(str(art_count))
            r = requests.get(url)
            data = r.text
            soup = BeautifulSoup(data, "html.parser")
            soup_abstract = soup.find(id = "abstract-body")
            abstract_ls.append(soup_abstract.text)
            time.sleep(.25)
        # sleep in between each journal so webpage doesn't hate me too much
        time.sleep(40) 
        print("resting for a moment so webpage doesn't hate me too much")
    return abstract_ls
    

In [ ]:
'''
    **************************************************************************
    Finding the top 10 or so Econ journal abstracts in terms of impact factor
    according to https://ideas.repec.org.
    
    Runtime: estimated to be 21 mins (1.5 minutes per journal, 14 journals including
    occasional links to the same journal at different times)
    **************************************************************************

note: output cleared for brevity.
'''
abstract_ls = get_abstracts(journ_ls)

In [ ]:
# get rid entries where abstracts are not available
for entry in abstract_ls:
    if entry == 'No abstract is available for this item.':
        abstract_ls.remove(entry)
        
len(abstract_ls)

In [ ]:
# save list of abstracts to dataframe
abstract_df = pd.DataFrame(abstract_ls, columns=["abstract"])
abstract_df.head()

In [ ]:
# save to csv file 
abstract_df.to_csv(r'raw_data_econ/topJournals_abstract.csv')

### Getting journals rank 11-20, in case we need it

In [29]:
''' Getting journals url, including same journals at multiple time 
periods as laid out in the webpage. '''
number = 0
journ_ls = []
for entry in range(14, 27):
    number = number + 1
    base_link = soup_href[entry]["href"]
    url = base_url + base_link
    print(number, url)
    journ_ls.append(url)
    

1 https://ideas.repec.org/s/aea/aecrev.html
2 https://ideas.repec.org/s/bin/bpeajo.html
3 https://ideas.repec.org/s/tpr/restat.html
4 https://ideas.repec.org/s/aea/jecper.html
5 https://ideas.repec.org/s/oup/rfinst.html
6 https://ideas.repec.org/s/kap/jecgro.html
7 https://ideas.repec.org/s/eee/moneco.html
8 https://ideas.repec.org/s/eee/crcspp.html
9 https://ideas.repec.org/s/eee/jfinec.html
10 https://ideas.repec.org/s/anr/reveco.html
11 https://ideas.repec.org/s/now/fnteco.html
12 https://ideas.repec.org/s/wly/econjl.html
13 https://ideas.repec.org/s/ecj/econjl.html


In [ ]:
'''
*********************************************
Getting journals rank 11-20 by impact factors over 10 years.

Estimated run time: 19.5 minutes (now set sleep to .25 seconds between articles
(primarily 200 articles per journal) and 40 seconds between journals (13 nondistinct))
note: output cleared for brevity.
*********************************************
'''
abstract_ls = get_abstracts(journ_ls)

In [ ]:
# get rid entries where abstracts are not available
for entry in abstract_ls:
    if entry == 'No abstract is available for this item.':
        abstract_ls.remove(entry)
        
# len(abstract_ls)

In [32]:
# save list of abstracts to dataframe
abstract_df = pd.DataFrame(abstract_ls, columns=["abstract"])
abstract_df.head()

,abstract
0,How large are the benefits of transportation i...
1,Macro developments leading up to the 2008 cris...
2,"This paper provides a simple, yet robust frame..."
3,We establish the importance of team-specific c...
4,We document substantial increases in agricultu...


In [33]:
# save to csv file 
abstract_df.to_csv(r'raw_data_econ/hiRankJournals_abstract.csv')

### Getting the "Not top journals" abstracts

In [7]:
'''
    For the journals not at the top, random sampling is used to generate a list of
    ranks between 20 and 100, with the 127th web scraped href being a link to rank 100. 
'''
np.random.seed(42)
random_ranks = np.random.randint(20, 127, 10)
random_ranks

array([122,  71, 112,  34, 126,  91,  80,  40, 122, 102])

In [28]:
number = 0
journ_ls = []
for entry in random_ranks:
    number = number + 1
    base_link = soup_href[entry]["href"]
    url = base_url + base_link
    print(number, url)
    journ_ls.append(url)
 

1 https://ideas.repec.org/s/bla/germec.html
2 https://ideas.repec.org/s/aea/aejmic.html
3 https://ideas.repec.org/s/ucp/ecdecc.html
4 https://ideas.repec.org/s/ucp/jlabec.html
5 https://ideas.repec.org/s/pia/review.html
6 https://ideas.repec.org/s/aen/journl.html
7 https://ideas.repec.org/s/cup/jfinqa.html
8 https://ideas.repec.org/s/mcb/jmoncb.html
9 https://ideas.repec.org/s/bla/germec.html
10 https://ideas.repec.org/s/ekn/ekonom.html


In [ ]:
'''
*********************************************
Getting random sample of journals rank 20-100 by impact factors over 10 years.

Estimated run time: 19.5 minutes (now set sleep to .25 seconds between articles
(primarily 200 articles per journal) and 40 seconds between journals (13 nondistinct))
note: output cleared for brevity.
*********************************************
'''
abstract_ls = get_abstracts(journ_ls)

In [68]:
# get rid entries where abstracts are not available
for entry in abstract_ls:
    if entry == 'No abstract is available for this item.':
        abstract_ls.remove(entry)
    if entry[0:12] == ' type="main"':
        entry = entry[41:]
        
len(abstract_ls)

1336

In [75]:
# save list of abstracts to dataframe
abstract_df = pd.DataFrame(abstract_ls, columns=["abstract"])
abstract_df.head()

,abstract
0,We conducted six treatments of a standard mora...
1,This paper analyzes blindfolded versus informe...
2,We analyze the drivers of the size of the audi...
3,Japan has been in a benign liquidity trap sinc...
4,This paper addresses tax loopholes that allow ...


In [76]:
# get rid of beginnings which start with ' type="main"'
for index, row in abstract_df.iterrows():
    if row["abstract"][0:12] == ' type="main"':
        row["abstract"] = row["abstract"][41:]

abstract_df.iloc[10]

abstract    We study how subjects in an experiment use dif...
Name: 10, dtype: object

In [78]:
# save to csv file 
abstract_df.to_csv(r'raw_data_econ/notHiJournals_abstract.csv')

#### More data for not top rank journals

In [31]:
'''
    For the journals not at the top, random sampling is used to generate a list of
    ranks between 20 and 100, with the 127th web scraped href being a link to rank 100. 
'''
import random
random.seed(42)
random_ranks2 = random.sample(range(20, 127), 20)

for item in random_ranks2:
    if item in random_ranks:
        random_ranks2.remove(item)
        
random_ranks2

[101, 23, 114, 55, 51, 48, 37, 33, 106, 89, 31, 95, 74, 24, 47, 49, 84, 97]

In [33]:
number = 0
journ_ls = []
for entry in random_ranks2:
    number = number + 1
    base_link = soup_href[entry]["href"]
    url = base_url + base_link
    print(number, url)
    journ_ls.append(url)
 

1 https://ideas.repec.org/s/fip/fedaer.html
2 https://ideas.repec.org/s/anr/reveco.html
3 https://ideas.repec.org/s/anr/reseco.html
4 https://ideas.repec.org/s/bla/randje.html
5 https://ideas.repec.org/s/wly/quante.html
6 https://ideas.repec.org/s/ijc/ijcjou.html
7 https://ideas.repec.org/s/wly/japmet.html
8 https://ideas.repec.org/s/uwp/jhriss.html
9 https://ideas.repec.org/s/bla/jemstr.html
10 https://ideas.repec.org/s/spr/weltar.html
11 https://ideas.repec.org/s/pal/imfstp.html
12 https://ideas.repec.org/s/fip/fednci.html
13 https://ideas.repec.org/s/fip/fednep.html
14 https://ideas.repec.org/s/now/fnteco.html
15 https://ideas.repec.org/s/bes/jnlbes.html
16 https://ideas.repec.org/s/eee/juecon.html
17 https://ideas.repec.org/s/oup/revfin.html
18 https://ideas.repec.org/s/now/jnlcfr.html


In [34]:
'''
*********************************************
Getting random sample of ~20 nondistinct journals rank 20-100 by impact factors over 10 years.

Estimated run time: 39 minutes (now set sleep to .25 seconds between articles
(primarily 200 articles per journal) and 40 seconds between journals (~20 nondistinct))

*********************************************
'''
abstract_ls = get_abstracts(journ_ls)

1 https://ideas.repec.org/a/fip/fedaer/y2010nv.95no.1.html
2 https://ideas.repec.org/a/fip/fedaer/y2010nv.95no.2.html
3 https://ideas.repec.org/a/fip/fedaer/y2010nv.95no.3.html
4 https://ideas.repec.org/a/fip/fedaer/y2009p1-12nv.94no.1.html
5 https://ideas.repec.org/a/fip/fedaer/y2009p1-14nv.94no.2.html
6 https://ideas.repec.org/a/fip/fedaer/y2009nv.94no.3.html
7 https://ideas.repec.org/a/fip/fedaer/y2009nv.94no.4.html
8 https://ideas.repec.org/a/fip/fedaer/y2008nv.93no.1.html
9 https://ideas.repec.org/a/fip/fedaer/y2008nv.93no.2.html
10 https://ideas.repec.org/a/fip/fedaer/y2008nv.93no.3.html
11 https://ideas.repec.org/a/fip/fedaer/y2008nv.93no.4.html
12 https://ideas.repec.org/a/fip/fedaer/y2007iq4pv-viinv.92no.4.html
13 https://ideas.repec.org/a/fip/fedaer/y2007iq4p1-24nv.92no.4.html
14 https://ideas.repec.org/a/fip/fedaer/y2007iq4p25-41nv.92no.4.html
15 https://ideas.repec.org/a/fip/fedaer/y2007iq4p43-69nv.92no.4.html
16 https://ideas.repec.org/a/fip/fedaer/y2007iq1-2p1-23nv.92nos.

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
resting for a moment so webpage doesn't hate me too much
1 https://ideas.repec.org/a/anr/reveco/v9y2017p1-20.html
2 https://ideas.repec.org/a/anr/reveco/v9y2017p21-58.html
3 https://ideas.repec.org/a/anr/reveco/v9y2017p59-85.html
4 https://ideas.repec.org/a/anr/reveco/v9y2017p87-101.html
5 https://ideas.repec

203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
resting for a moment so webpage doesn't hate me too much
1 https://ideas.repec.org/a/anr/reveco/v9y2017p1-12.html
2 https://ideas.repec.org/a/anr/reveco/v9y2017p13-31.html
3 https://ideas.repec.org/a/anr/reve

402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
resting for a moment so webpage doesn't hate me too much
1 https://ideas.repec.org/a/bla/randje/v48y2017i4p877-905.html
2 https://ideas.repec.org/a/bla/randje/v48y2017i4p906-926.html
3 https://ideas.repec

602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
resting for a moment so webpage doesn't hate me too much
1 https://ideas.repec.org/a/wly/quante/v8y2017i3p685-728.html
2 https://ideas.repec.org/a/wly/quante/v8y2017i3p729-760.html
3 https://ideas.repec.o

...

2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
resti

2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
resti

2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
resti

2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871
2872
2873
2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
resti

2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
resti

In [51]:
# save list of abstracts to dataframe
abstract_df = pd.DataFrame(abstract_ls, columns=["abstract"])
# add 0 for top_journal column dummy 
abstract_df['top_journal'] = 0


In [68]:
# get rid of beginnings which start with ' type="main"'
for index, row in abstract_df.iterrows():
    if row["abstract"][0:12] == ' type="main"':
        string = row["abstract"][14:]
        abstract_df.loc[index, "abstract"] = string
        
abstract_df["abstract"].iloc[727]

"Using an agency model, we show how delegation, by generating additional private information, improves dynamic incentives under limited commitment. It circumvents ratchet effects and facilitates the revelation of persistent private information through two effects: a play-hardball effect, which mitigates an efficient agent's ratchet incentive, and a carrot effect which reduces an inefficient agent's take-the-money-and-run incentive. Although delegation entails a loss of control, it is optimal when uncertainty about operational efficiency is large. Moreover, delegation is more effective with production complementarity. We also consider different modes of commitment to yield insights into optimal organizational boundaries."

In [69]:
# get rid entries where abstracts are not available
for index, row in abstract_df.iterrows():
    if row["abstract"] == 'No abstract is available for this item.':
        abstract_df.drop(index, inplace=True)
        
abstract_df.head()

,abstract,top_journal
0,This reprint of a 1993 article outlines what C...,0
1,"In 2007, the United States began to experience...",0
2,In response to the wave of residential mortgag...,0
3,"For developing countries, remittances are an i...",0
4,"For the past half-century, a debate has raged ...",0


In [70]:
# save to csv file 
abstract_df.to_csv(r'raw_data_econ/notHiJournals_2_abstract.csv')